In [1]:
from pathlib import Path
import pandas as pd

# aktuelles Arbeitsverzeichnis vom Notebook
ROOT = Path.cwd().parent   # von notebooks/ nach project/

# Datenpfad zusammensetzen
DATA_DIR = ROOT / "data" / "202507-divvy-tripdata"
csv_file = DATA_DIR / "202507-divvy-tripdata.csv"

df = pd.read_csv(csv_file)

In [2]:
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,455D43BD91D73437,classic_bike,2025-07-05 17:15:08.456,2025-07-05 17:25:47.079,Lincoln Ave & Diversey Pkwy,CHI00285,Lincoln Ave & Addison St,CHI00478,41.932225,-87.658617,41.946176,-87.673308,member
1,9D4A6B723ECD98CA,classic_bike,2025-07-01 13:57:38.878,2025-07-01 14:06:35.780,Cottage Grove Ave & Oakwood Blvd,CHI00400,Cottage Grove Ave & 47th St,CHI00488,41.822985,-87.607100,41.809855,-87.606755,member
2,C57044CF523302ED,classic_bike,2025-07-31 16:49:28.142,2025-07-31 17:15:28.999,Theater on the Lake,CHI00420,Winthrop Ave & Lawrence Ave,CHI00391,41.926277,-87.630834,41.968812,-87.657659,member
3,AFD35552E6685B6E,electric_bike,2025-07-17 09:36:21.058,2025-07-17 09:46:54.706,Pine Grove Ave & Waveland Ave,CHI00354,Winthrop Ave & Lawrence Ave,CHI00391,41.949473,-87.646453,41.968812,-87.657659,member
4,C0582EBAA6CED519,classic_bike,2025-07-02 18:43:45.213,2025-07-02 18:57:06.687,Theater on the Lake,CHI00420,Sheffield Ave & Wellington Ave,CHI00274,41.926277,-87.630834,41.936253,-87.652662,member


In [3]:
df.shape

(763432, 13)

In [4]:
tmp = pd.concat([
    pd.DataFrame({"station_id": df["start_station_id"], "delta":  1}),
    pd.DataFrame({"station_id": df["end_station_id"],   "delta": -1}),
], ignore_index=True)

result = (
    tmp.groupby("station_id", as_index=False)["delta"].sum()
       .rename(columns={"delta": "difference"})
       .sort_values("difference", ascending=False))
print(result.head())
result.tail()

     station_id  difference
269    CHI00483         283
1098   CHI01745         253
131    CHI00345         245
1374   CHI02025         233
81     CHI00294         228


,station_id,difference
310,CHI00525,-219
457,CHI00674,-233
1424,CHI02078,-238
235,CHI00449,-325
1097,CHI01744,-382


In [6]:
# Datumsspalten parsen
df["started_at"] = pd.to_datetime(df["started_at"])
df["ended_at"] = pd.to_datetime(df["ended_at"])

# Tagesweise Bewegungen erfassen
tmp = pd.concat([
    pd.DataFrame({
        "station_id": df["start_station_id"],
        "date": df["started_at"].dt.date,
        "delta": 1
    }),
    pd.DataFrame({
        "station_id": df["end_station_id"],
        "date": df["ended_at"].dt.date,
        "delta": -1
    })
], ignore_index=True)

# Gruppieren nach Tag und Station
daily_balance = (
    tmp.groupby(["date", "station_id"], as_index=False)["delta"].sum()
       .rename(columns={"delta": "difference"})
)

# Beispiel: Tag auswählen
day = pd.to_datetime("2025-07-05").date()

# Stationsbewegungen dieses Tages
day_data = daily_balance[daily_balance["date"] == day]

# Top 5 mit meisten Abgängen und Zugängen
most_added = day_data.sort_values("difference", ascending=False).head(5)
most_removed = day_data.sort_values("difference", ascending=True).head(5)

print(f"🚲 {day}: Meiste Räder hinzugekommen:")
print(most_added)
print("\n🚲 {day}: Meiste Räder abgeholt:")
print(most_removed)


🚲 2025-07-05: Meiste Räder hinzugekommen:
            date station_id  difference
3634  2025-07-05   CHI00216          35
3695  2025-07-05   CHI00279          19
3930  2025-07-05   CHI00521          19
3844  2025-07-05   CHI00432          18
4100  2025-07-05   CHI00761          17

🚲 {day}: Meiste Räder abgeholt:
            date station_id  difference
3934  2025-07-05   CHI00525         -66
3832  2025-07-05   CHI00420         -49
3839  2025-07-05   CHI00427         -24
3709  2025-07-05   CHI00295         -23
3760  2025-07-05   CHI00347         -22
